<a href="https://colab.research.google.com/github/SurajitBiswasPhysics/surajit-biswas.github.io/blob/main/fcc_sms_text_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# import libraries
try:
  # %tensorflow_version only exists in Colab.
  !pip install tf-nightly
except Exception:
  pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, TextVectorization, Embedding, GlobalAveragePooling1D, Dense



# Load Data
train_df = pd.read_csv("train-data.tsv", sep='\t', header=None, names=['label', 'message'])
test_df = pd.read_csv("valid-data.tsv", sep='\t', header=None, names=['label', 'message'])

# Preprocess Labels: Map 'ham' to 0 and 'spam' to 1
train_df['label'] = train_df['label'].map({'ham': 0, 'spam': 1})
test_df['label'] = test_df['label'].map({'ham': 0, 'spam': 1})

# Extract Features and Labels
X_train, y_train = train_df['message'].astype(str).values, train_df['label'].values
X_test, y_test = test_df['message'].astype(str).values, test_df['label'].values

print("Training Data Shape:", X_train.shape, y_train.shape)
print("Test Data Shape:", X_test.shape, y_test.shape)

# Text Vectorization Layer
vectorizer = TextVectorization(
    max_tokens=10000,                   # Vocabulary size
    output_sequence_length=100,         # Max tokens per message
    standardize="lower_and_strip_punctuation"
)

# Adapt the Vectorizer to Training Data
vectorizer.adapt(X_train)

# Build the Model
model = Sequential([
    Input(shape=(1,), dtype=tf.string),  # Accepts raw text input
    vectorizer,                         # Text preprocessing
    Embedding(input_dim=10000, output_dim=16),
    GlobalAveragePooling1D(),
    Dense(16, activation='relu'),
    Dense(1, activation='sigmoid')      # Binary classification (spam/ham)
])

# Compile the Model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Model Summary
model.summary()

# Train the Model
history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=10,
    batch_size=32
)

In [ ]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):


  pred_text_tensor = tf.constant([pred_text])  # Batch of size 1

  # Predict probability
  prediction_prob = model.predict(pred_text_tensor, verbose=0)[0][0]

  # Map probability to label
  predicted_label = 'spam' if prediction_prob > 0.5 else 'ham'

  prediction = [float(prediction_prob), predicted_label]

  return (prediction)

pred_text = "how are you doing today?"

prediction = predict_message(pred_text)
print(prediction)

In [ ]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()
